In [2]:
import os
import pandas as pd
import numpy as np
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import precision_score, confusion_matrix, classification_report

In [3]:
df = pd.read_csv('/home/jensk/code/JensKlug/zillows_real_estate/raw_data/HouseTS.csv')

df = df.sort_values(['city', 'zipcode', 'date']).reset_index(drop=True)
df['date'] = pd.to_datetime(df['date'])

train = df[df.date.dt.year<2023]
test = df[df.date.dt.year>2022]

grouped_train = train.groupby(['city', 'zipcode'])
grouped_test = test.groupby(['city', 'zipcode'])

In [4]:
'''
ts_train = pd.DataFrame()
ts_test = pd.DataFrame()


ls = [1,3,6,12]

i = 2

#for i in ls:
ts_train[f'target_profitable_{i}m'] = (grouped_train['price'].shift(-i) > 1.02 * grouped_train['price'].shift(0)).astype(int) # We fail to exclude the last three obervastions since with the shaft they are NaN!
ts_train['zipcode'] = grouped_train['zipcode'].shift(0).astype('object') # but remains still an integer...


numeric_column_names = train.select_dtypes(include='number').columns # drop the lags of zipcode

for feature in numeric_column_names:
    for lag in range(1, 4):  # 1 to 3 months
        ts_train[f'{feature}_{lag}m'] = grouped_train[f'{feature}'].shift(lag)
'''

"\nts_train = pd.DataFrame()\nts_test = pd.DataFrame()\n\n\nls = [1,3,6,12]\n\ni = 2\n\n#for i in ls:\nts_train[f'target_profitable_{i}m'] = (grouped_train['price'].shift(-i) > 1.02 * grouped_train['price'].shift(0)).astype(int) # We fail to exclude the last three obervastions since with the shaft they are NaN!\nts_train['zipcode'] = grouped_train['zipcode'].shift(0).astype('object') # but remains still an integer...\n\n\nnumeric_column_names = train.select_dtypes(include='number').columns # drop the lags of zipcode\n\nfor feature in numeric_column_names:\n    for lag in range(1, 4):  # 1 to 3 months\n        ts_train[f'{feature}_{lag}m'] = grouped_train[f'{feature}'].shift(lag)\n"

In [5]:
#ts_train #type(ts_train['zipcode'][0])

In [6]:
ts_train = pd.DataFrame()
ts_test = pd.DataFrame()


ls = [1,3,6,12]

i = 1

#for i in ls:
ts_train[f'target_profitable_{i}m'] = (grouped_train['price'].shift(-i) > 1.02 * grouped_train['price'].shift(0)).astype(int) # We fail to exclude the last three obervastions since with the shaft they are NaN!
ts_train['zipcode'] = grouped_train['zipcode'].shift(0)# .astype('object') # but remains still an integer...
numeric_column_names = train.select_dtypes(include='number').columns # drop the lags of zipcode

for feature in numeric_column_names:
    for lag in range(1, 4):  # 1 to 3 months
        ts_train[f'{feature}_{lag}m'] = grouped_train[f'{feature}'].shift(lag)

ts_train = ts_train.dropna().reset_index(drop=True)

X_train = ts_train.drop(columns=[f'target_profitable_{i}m'])
y_train = ts_train[f'target_profitable_{i}m']


ts_test[f'target_profitable_{i}m'] = (grouped_test['price'].shift(-i) > 1.02 * grouped_test['price'].shift(0)).astype(int)
ts_test['zipcode'] = grouped_test['zipcode'].shift(0)# .astype('object') # but remains still an integer...
numeric_column_names = test.select_dtypes(include='number').columns # drop the lags of zipcode

for feature in numeric_column_names:
    for lag in range(1, 4):  # 1 to 6 months
        ts_test[f'{feature}_{lag}m'] = grouped_test[f'{feature}'].shift(lag)

ts_test = ts_test.dropna().reset_index(drop=True)

X_test = ts_test.drop(columns=[f'target_profitable_{i}m'])
y_test = ts_test[f'target_profitable_{i}m']

scaler = MinMaxScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

model = SGDClassifier(loss='log_loss')
model.fit(X_train_scaled, y_train)

y_pred = model.predict(X_test_scaled)

np.savetxt(f"output_{i}m.csv", y_pred, delimiter=",", header=f"target_-{i}m", comments='')

del y_pred


/tmp/ipykernel_42416/2567415700.py:16: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  ts_train[f'{feature}_{lag}m'] = grouped_train[f'{feature}'].shift(lag)
/tmp/ipykernel_42416/2567415700.py:16: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  ts_train[f'{feature}_{lag}m'] = grouped_train[f'{feature}'].shift(lag)
/tmp/ipykernel_42416/2567415700.py:16: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once

In [17]:
zipcode_identifier = ts_test['zipcode']
zipcode_identifier

0        30002
1        30002
2        30002
3        30002
4        30002
         ...  
56029    34698
56030    34698
56031    34698
56032    34698
56033    34698
Name: zipcode, Length: 56034, dtype: int64

In [8]:
#X_test_scaled

In [9]:
os.getcwd()

'/home/jensk/code/JensKlug/zillows_real_estate/notebooks'

In [10]:
output_1 = pd.read_csv('output_1m.csv')
output_1

,target_-1m
0,0.0
1,0.0
2,0.0
3,0.0
4,0.0
...,...
56029,0.0
56030,0.0
56031,0.0
56032,0.0


In [11]:
output_2 = pd.read_csv('output_2m.csv')
output_2

,target_-2m
0,0.0
1,0.0
2,0.0
3,0.0
4,0.0
...,...
56029,0.0
56030,0.0
56031,0.0
56032,0.0


In [18]:
df_combined = pd.concat([zipcode_identifier, output_1, output_2], axis=1)
df_combined

,zipcode,target_-1m,target_-2m
0,30002,0.0,0.0
1,30002,0.0,0.0
2,30002,0.0,0.0
3,30002,0.0,0.0
4,30002,0.0,0.0
...,...,...,...
56029,34698,0.0,0.0
56030,34698,0.0,0.0
56031,34698,0.0,0.0
56032,34698,0.0,0.0


In [21]:
grouped_result = df_combined.groupby(['zipcode']).mean()
grouped_result.value_counts()

target_-1m  target_-2m
0.000000    0.000000      5227
            0.111111       279
            0.222222       155
            0.333333       143
            0.444444        95
            0.555556        84
            0.666667        72
            1.000000        65
            0.777778        56
            0.888889        43
0.111111    0.555556         4
            0.222222         1
            0.777778         1
            0.888889         1
Name: count, dtype: int64

In [13]:
'''ts_train = pd.DataFrame()
ts_test = pd.DataFrame()

#ls = [1,3,6,12]



#for i in ls:
ts_train['target_profitable_3m'] = (grouped_train['price'].shift(-6) > 1.02 * grouped_train['price'].shift(0)).astype(int) # We fail to exclude the last three obervastions since with the shaft they are NaN!

numeric_column_names = train.select_dtypes(include='number').columns # drop the lags of zipcode

for feature in numeric_column_names:
    for lag in range(1, 4):  # 1 to 3 months
        ts_train[f'{feature}_{lag}m'] = grouped_train[f'{feature}'].shift(lag)

ts_train = ts_train.dropna().reset_index(drop=True)

X_train = ts_train.drop(columns=['target_profitable_3m'])
y_train = ts_train['target_profitable_3m']


ts_test['target_profitable_3m'] = (grouped_test['price'].shift(-6) > 1.02 * grouped_test['price'].shift(0)).astype(int)

numeric_column_names = test.select_dtypes(include='number').columns # drop the lags of zipcode

for feature in numeric_column_names:
    for lag in range(1, 4):  # 1 to 6 months
        ts_test[f'{feature}_{lag}m'] = grouped_test[f'{feature}'].shift(lag)

ts_test = ts_test.dropna().reset_index(drop=True)

X_test = ts_test.drop(columns=['target_profitable_3m'])
y_test = ts_test['target_profitable_3m']

scaler = MinMaxScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

model = SGDClassifier(loss='log_loss')
model.fit(X_train_scaled, y_train)

y_pred = model.predict(X_test_scaled)
'''

"ts_train = pd.DataFrame()\nts_test = pd.DataFrame()\n\n#ls = [1,3,6,12]\n\n\n\n#for i in ls:\nts_train['target_profitable_3m'] = (grouped_train['price'].shift(-6) > 1.02 * grouped_train['price'].shift(0)).astype(int) # We fail to exclude the last three obervastions since with the shaft they are NaN!\n\nnumeric_column_names = train.select_dtypes(include='number').columns # drop the lags of zipcode\n\nfor feature in numeric_column_names:\n    for lag in range(1, 4):  # 1 to 3 months\n        ts_train[f'{feature}_{lag}m'] = grouped_train[f'{feature}'].shift(lag)\n\nts_train = ts_train.dropna().reset_index(drop=True)\n\nX_train = ts_train.drop(columns=['target_profitable_3m'])\ny_train = ts_train['target_profitable_3m']\n\n\nts_test['target_profitable_3m'] = (grouped_test['price'].shift(-6) > 1.02 * grouped_test['price'].shift(0)).astype(int)\n\nnumeric_column_names = test.select_dtypes(include='number').columns # drop the lags of zipcode\n\nfor feature in numeric_column_names:\n    for l

In [14]:
'''ts_train = pd.DataFrame()
ts_test = pd.DataFrame()


ls = [1,3,6,12]

for i in ls:
    ts_train[f'target_profitable_{i}m'] = (grouped_train['price'].shift(-i) > 1.02 * grouped_train['price'].shift(0)).astype(int) # We fail to exclude the last three obervastions since with the shaft they are NaN!

    numeric_column_names = train.select_dtypes(include='number').columns # drop the lags of zipcode

    for feature in numeric_column_names:
        for lag in range(1, 4):  # 1 to 3 months
            ts_train[f'{feature}_{lag}m'] = grouped_train[f'{feature}'].shift(lag)

    ts_train = ts_train.dropna().reset_index(drop=True)

    X_train = ts_train.drop(columns=[f'target_profitable_{i}m'])
    y_train = ts_train[f'target_profitable_{i}m']


    ts_test[f'target_profitable_{i}m'] = (grouped_test['price'].shift(-i) > 1.02 * grouped_test['price'].shift(0)).astype(int)

    numeric_column_names = test.select_dtypes(include='number').columns # drop the lags of zipcode

    for feature in numeric_column_names:
        for lag in range(1, 4):  # 1 to 6 months
            ts_test[f'{feature}_{lag}m'] = grouped_test[f'{feature}'].shift(lag)

    ts_test = ts_test.dropna().reset_index(drop=True)

    X_test = ts_test.drop(columns=[f'target_profitable_{i}m'])
    y_test = ts_test[f'target_profitable_{i}m']

    scaler = MinMaxScaler()
    scaler.fit(X_train)
    X_train_scaled = scaler.transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    model = SGDClassifier(loss='log_loss')
    model.fit(X_train_scaled, y_train)

    y_pred = model.predict(X_test_scaled)

    np.savetxt(f"output_{i}m.csv", y_pred, delimiter=",", header=f"target_-{i}m", comments='')

    del y_pred
'''

'ts_train = pd.DataFrame()\nts_test = pd.DataFrame()\n\n\nls = [1,3,6,12]\n\nfor i in ls:\n    ts_train[f\'target_profitable_{i}m\'] = (grouped_train[\'price\'].shift(-i) > 1.02 * grouped_train[\'price\'].shift(0)).astype(int) # We fail to exclude the last three obervastions since with the shaft they are NaN!\n\n    numeric_column_names = train.select_dtypes(include=\'number\').columns # drop the lags of zipcode\n\n    for feature in numeric_column_names:\n        for lag in range(1, 4):  # 1 to 3 months\n            ts_train[f\'{feature}_{lag}m\'] = grouped_train[f\'{feature}\'].shift(lag)\n\n    ts_train = ts_train.dropna().reset_index(drop=True)\n\n    X_train = ts_train.drop(columns=[f\'target_profitable_{i}m\'])\n    y_train = ts_train[f\'target_profitable_{i}m\']\n\n\n    ts_test[f\'target_profitable_{i}m\'] = (grouped_test[\'price\'].shift(-i) > 1.02 * grouped_test[\'price\'].shift(0)).astype(int)\n\n    numeric_column_names = test.select_dtypes(include=\'number\').columns # dro